In [57]:
import pandas as pd
import pyrfume
from pyrfume.odorants import get_cids
import rdkit
from rdkit import Chem
import re

In [60]:
re.sub('l-(?![0-9])', 'l ', 'amyl-2-acetate')

'amyl-2-acetate'

In [91]:
from pathlib import Path
p = Path(pyrfume.DATA)
old_names = []
new_names = []
for file in (p / 'leon' / '3D_mol_files').glob('*.mol'):
    name = file.name.replace('.mol', '')
    old_names.append(name)
    # Replace underscore with hyphen
    name = name.replace('_','-')
    # Remove extraneous hyphens 
    name = re.sub('(?<![0-9\(])-(?![0-9])', ' ', name)
    # Add back hyphens after prefixes
    for x in ['alpha', 'beta', 'gamma', 'delta', 'tert', 'L', 'D', 'm', 'o', 'p', 'cis', 'trans', 'sec']:
        name = name.replace('%s '%x, '%s-'%x)
    # Move isomeric identifiers to the front of the name
    for x in ['(-)', '(+)']:
        if x in name:
            name = '%s-%s' % (x, name.replace(x, ''))
    new_names.append(name)
    #print(name)

In [92]:
cids = get_cids(new_names)

[-----------------------100%---------------------] 268 out of 268 complete               
Could not find phenethyl-2-furoate
Could not find celeriax
Could not find methyl-2-octynoate
Could not find methyl-3-methylbutenoate
Could not find 2,5-dimethyl-4-hydroxy-3(2H)furanone


Could not find 2,5-dimethyl-4-hydroxy-3(2H)furanone
Could not find methyl-3-methylbutenoate
Could not find methyl-2-octynoate
Could not find celeriax
Could not find phenethyl-2-furoate


In [95]:
df = pd.Series(cids, name='CID').to_frame()
df['Old Name'] = old_names
df.index.name = 'Name'
df = df.reset_index()
df.head()

,Name,CID,Old Name
0,alpha-angelica lactone,11559,alpha-angelica_lactone
1,nonanal,31289,nonanal
2,mesityl oxide,8858,mesityl_oxide
3,trans-2-hexenoic acid,5282707,trans-2-hexenoic_acid
4,2-octanone,8093,2-octanone


In [96]:
df[df['CID']==0]

,Name,CID,Old Name
67,"2,5-dimethyl-4-hydroxy-3(2H)furanone",0,"2,5-dimethyl-4-hydroxy-3(2H)furanone"
76,methyl-3-methylbutenoate,0,methyl_3-methylbutenoate
79,methyl-2-octynoate,0,methyl_2-octynoate
155,celeriax,0,celeriax
170,phenethyl-2-furoate,0,phenethyl_2-furoate


In [97]:
df.loc[67, 'CID'] = 19309
df.loc[76, 'CID'] = 11160 
df.loc[79, 'CID'] = 8092 
df.loc[155, 'CID'] = 28500  
df.loc[170, 'CID'] = 251531

In [99]:
df = df.set_index('CID').sort_index()
df.head()

In [104]:
df.to_csv(p / 'leon' / 'leon_cids.csv', sep='\t')